# Exploring Sentence, Chunk, and Entity Embeddings for Retrieval  


This notebook provides a **proof of concept (PoC)** for working with embeddings in the *Impresso* corpus across three levels of granularity:  

1. **Sentence embeddings** – fine-grained retrieval at the level of individual sentences (e.g., `lepetitparisien-1912-11-13-a-i0001-s-11`).  
2. **Chunk embeddings** – broader retrieval at the level of aggregated text chunks (e.g., `lepetitparisien-1912-11-13-a-i0001-c-1`).  
3. **Entity embeddings** – retrieval of linked entities (e.g., `Q380083` or `Jonas Furrer`).  

We refer to sentences and chunks as _subdocuments_ and we test these *subdoc* embeddings with two complementary query scenarios:  

- **In-corpus queries** – selecting a query directly from the *Impresso* corpus.  
- **Out-of-corpus queries** – embedding an external query (e.g., manually formulated or from another source).  

For the purpose of this PoC:

👉 The **subdocs** = sentence and chunk embeddings for **front pages** of all newspapers in **1912 (Titanic)** and **1986 (Tchernobyl)**.  
👉 The **entities** = entity embeddings for **person entities** in the years [+/- 5y] around the same years.

From now on, we refer to this PoC as a set of **experiments**. The experiments use direct queries to our internal retrieval system ([Solr](https://solr.apache.org/)).

Let's first connect to Impresso:

In [1]:
from impresso import connect

impresso = connect('https://dev.impresso-project.ch/public-api/v1')

🎉 You are now connected to the Impresso API!  🎉
🔗 Using API: https://dev.impresso-project.ch/public-api/v1


## Get all experiments

In [2]:
impresso.experiments.find()

,name,description
id,,
subdoc-embeddings,Experiment with sentence and character level e...,\n Generates embeddings for subdocuments usin...
entity-profiles,Experiment with entity profiles and their embe...,\n Generates embeddings for subdocuments usin...


### Subdoc Embeddings Experiments

#### Sentence Embeddings - In-corpus queries

Let's search for some documents, take their embeddings and then search by embedding in Impresso.

In [3]:
sentence = "Le congrès international s'est tenu à Paris pour discuter des avancées scientifiques de la décennie."

result = impresso.experiments.execute(
    experiment_id="subdoc-embeddings",
    body={
        "solrPayload": {
            "query": f"content_txt_fr:({sentence}) AND type_s:s", # type_s:s restricts search to sentences
            "limit": 5,
            "params": {"hl": False}
        }
    }
)

docs = result["solrResponse"]["response"]["docs"]
print(f"✅ Got {len(docs)} Solr document(s)\n")


✅ Got 5 Solr document(s)



In [4]:
for i, d in enumerate(docs, 1):
    print(f"--- Result {i} ---")
    print(d.get("content_txt_fr", "[No text]"))

--- Result 1 ---
Mais c’ est une question qui ne peut se régler en congrès internationaux et c’ est pourquoi le pays cjui ne présente pas une natalité suffisante sera étranglé, ce qui ne sera d’ ailleurs qu’ une avance sur son sui- cide.
--- Result 2 ---
ÉDITION DE PARIS Les Eves nouvelles Les suffragettes françaises on * tenu, dimanche, une réunion où fut discutée l' intéressante question du vote municipal des femmes.
--- Result 3 ---
Le 20 juillet 1889, au Congrès socialiste intermfci mal de Paris, il proposa la résolution suivante : « Il sera organisé une grande manifestation internationale à dr.
--- Result 4 ---
Le congrès socialiste de 19 10 a réservé la question de principe, mais il avait stipulé que conformément aux résolutions des congrès internationaux de Paris et d’ Amsterdam, il n’ admettait pas comme possible la participation individuelle de certains socialistes, sans l’ assentiment du parti ouvrier, à un ministère quelconque ; le congrès de 1910 avait décidé, pour le surpl

In [26]:
list(docs[0].keys())

['id',
 'imp_ids_ss',
 'surfaces_ss',
 'ci_ids_ss',
 'mention_keys_ss',
 'ci_lg_s',
 'wiki_masterlabel_s',
 'wiki_url_s',
 'date_of_birth_dt',
 'date_of_death_dt',
 'wkd_occupations_ss',
 'wkd_occupation_qids_ss',
 'wkd_entity_types_ss',
 'wiki_summaries_t',
 'contexts_ss',
 'entity_mixed_emb_v768',
 'entity_encyc_emb_v768',
 'entity_media_emb_v768',
 '_version_',
 '_root_']

In [6]:
docs[0]['content_txt_fr'][:200]  # first 200 characters of the document content

'Mais c’ est une question qui ne peut se régler en congrès internationaux et c’ est pourquoi le pays cjui ne présente pas une natalité suffisante sera étranglé, ce qui ne sera d’ ailleurs qu’ une avanc'

Let's take the first returned document's embedding.

In [7]:
embedding = docs[0]['gte_multi_v768']

embedding[:3]

[-0.081427164, 0.064372316, -0.045108054]

In [8]:
embedding = result['solrResponse']['response']['docs'][0]['gte_multi_v768']

In [9]:
result = impresso.experiments.execute(
    experiment_id="subdoc-embeddings",
    body={
        "solrPayload": {
            "query": "{!knn f=gte_multi_v768 topK=3}" + str(embedding),
            "limit": 3,
            "params": {
                "fq": "type_s:s",   # type_s:s restricts search to sentences (s=sentence)
                # "fl": "id,score,content_txt_fr,ci_id_s", -- add these later if you want to return only specific fields
                # for now let's return everything
                "hl": False
            }
        }
    }
)

docs = result["solrResponse"]["response"]["docs"]
print(f"✅ Got {len(docs)} Solr document(s)\n")

✅ Got 3 Solr document(s)



In [10]:
# docs

In [11]:
for i, d in enumerate(docs):
    print(f"--- Result {i} ---")
    print(d.get("content_txt_fr", "[No text]"))
    print(d.get("content_txt_de", "[No text]"))

--- Result 0 ---
Mais c’ est une question qui ne peut se régler en congrès internationaux et c’ est pourquoi le pays cjui ne présente pas une natalité suffisante sera étranglé, ce qui ne sera d’ ailleurs qu’ une avance sur son sui- cide.
[No text]
--- Result 1 ---
[No text]
weil sie nicht die ganze Nation in dem Parlament vertreten sehen wolle.
--- Result 2 ---
[No text]
Es steht zu hoffen, daß damit die vom Bundesrate getanen völlig gewesen Schritte nicht nutzlos sind, und daß in nicht allzu ferner Zeit doch noch eine solche internationale Konferenz sich mit dein Problem beschäftigen wird.


#### Sentence Embeddings - Out-of-corpus queries


In [12]:
sentence = "Le congrès international s'est tenu à Paris pour discuter des avancées scientifiques de la décennie."

embedding = impresso.tools.embed_text(text=sentence, target="text")
embedding

'gte-768:SldQPEKgFj3hqBq9KbZDO44Ntr2m3B28HGQiPBZYHDzoyi89VZGjvdn/jD376Ia83lSSO/TikzyG1fu9RWKxPOTYkj0jF8k99JncPaxsarx18VI92R20PTlvrbxuzWS9A4uUO6lYFTsWTQ09+QKVvVQiXL3mpiE9FbOxvRhr/DyplVE8vIZ6PQ/ivj0660e90aF9PE0Yrzx12ZC9Awv/O0xEortNwYO8vr0rvTctNrtTsAu87z18PJPXGL2lOO08N3NSPORAgz1FtBY9YC4cPY3tFT0JM8g8aQ5zvHa067v9YS+9wol3vAzUtj2SRWG9jPGMO4I2M7uCBLy8qxM3PaXMYb3Rs1+9ukY5POf2lbycXIc9j/XyvM9A4L29ToA8lBeBPAsP6DwO+3i9y0m4PQqOAL3f1eC8NpOEPDLQpzzHMxK9MQc1PLpK9rwbOxI9EH4ZvZiKrrzGDFi8qVTbuztLPDr/IYQ9HwPZPcMXfT1SmI06B6WOvNNM9jwBFWo4X+24vJDvoL1biHC8vca5O6NWYbySuqW8KkXkvG8SirzGp+w8J0WSvCIjuLtRdjG9irEYvYnl8juWY/Y7iURPvEciEL1BHzm9gkebvSwx3Dz4smQ8f1lKPRGzYD0NzEa9ujSDOyduHLyrzYI78vxhvT/5xb24sBi9IgSzPBhvsr1Yufc8kyFIvYcaGT2KvtK69kKCPI9oD73T+bq8vFyDvW2jp7wwuTM9+FC6PFAzlbuWlns7sEdiPbj6ibxZYEI9Cr4dPMEeXT2Qsom8FObQvPubDT3o8xs9N69IOwMPsLtImW09uayXvFIUMDz8ssK7eXa7vIYX6zw2G3O9R5eKvCH+kz0plv48P516PSE+lb1heNW8rjr2O8RRrTvoQ8O84cOaPX0YbjwHJ008qCejPNlKFDy5BPO85f6RvDaqWzsYDvU8DCqAPck/ib15Z549f/jiO5MYiL2aEFi9g2uNvdMusDuMeWW

In [13]:
import base64
import struct

# convert base64 string to a float array
_, arr = embedding.split(':')
arr = base64.b64decode(arr)
outof_corpus_emb = [struct.unpack('f', arr[i:i+4])[0] for i in range(0, len(arr), 4)]

len(outof_corpus_emb)

768

In [14]:
outof_corpus_emb[:5]

[0.012716123834252357,
 0.0367739275097847,
 -0.037758711725473404,
 0.002986321458593011,
 -0.08889304101467133]

In [15]:
result = impresso.experiments.execute(
  experiment_id="subdoc-embeddings",
  body={
    "solrPayload": {
      "query": "{!knn f=gte_multi_v768 topK=3}" + str(outof_corpus_emb),
      "limit": 3,
      "params": {
        "hl": False
      }
    }
  }
)
docs = result["solrResponse"]["response"]["docs"]
print(f"✅ Got {len(docs)} Solr document(s)\n")

✅ Got 3 Solr document(s)



In [16]:
for i, d in enumerate(docs):
    print(f"--- Result {i} ---")
    print(d.get("content_txt_fr", "[No text]"))
    print(d.get("content_txt_de", "[No text]"))

--- Result 0 ---
[No text]
Bei der internationalen Delegierten konferenz der Notare han.
--- Result 1 ---
[No text]
l " " " erwähnten Kongreß « « « gesetzte internatio. " "
--- Result 2 ---
Paris, 10 janvier.
[No text]


#### Chunk Embeddings - In-corpus queries


In [17]:
chunk = (
    "Le congrès international s'est tenu à Paris pour discuter des avancées scientifiques de la décennie. "
    "Des chercheurs venus de nombreux pays ont présenté leurs travaux les plus récents dans les domaines de la physique, "
    "de la biologie et des sciences sociales. "
    "Les débats ont mis en lumière les progrès réalisés grâce à la collaboration entre institutions européennes et américaines, "
    "ainsi que les défis à venir pour une recherche plus ouverte et interdisciplinaire. "
    "La rencontre s’est conclue par l’adoption d’une résolution encourageant la diffusion libre des connaissances scientifiques.")

result = impresso.experiments.execute(
    experiment_id="subdoc-embeddings",
    body={
        "solrPayload": {
            "query": f"content_txt_fr:({chunk}) AND type_s:c",
            "limit": 5,
            "params": {"hl": False}
        }
    }
)

docs = result["solrResponse"]["response"]["docs"]
print(f"✅ Got {len(docs)} Solr document(s)\n")

✅ Got 5 Solr document(s)



In [18]:
for i, d in enumerate(docs):
    print(f"--- Result {i} ---")
    print(d.get("content_txt_fr", "[No text]"))
    print(d.get("content_txt_de", "[No text]"))

--- Result 0 ---
et leurs gouvernements respectifs. '.. ".'•••En outre, la nouvelle convention accorde la franchise de port pour la correspondance drdniaire des institutions nationales ayant un caractère scientifique et d' intérêt général ; ainsi qu' aux congrès scientifiques sud-américains composés de la majorité des pays de ce continent.Cette sage disposition constitue pour les sciences un bel encouragement et elle contribuera à répandre avec plus de facilité sur les immenses territoires de l' Amérique du Sud les progrès dus aux- efforts du génie humain.D' autre pari, le congrès postal dont nous analysons les travaux n' a pas blié les services rendus jpar la presse dans le domaine de l' éducation - des peuples.Il a cru devoir également favoriser la diffusion des idées, dans le louable désir d' accélérer - l' avènement de la turité - politique des citoyens.C' est pourquoi la franchise de.port est accordée aux éditeurs de journaux quotidiens et de publications périodiques sud-américain

In [19]:
embedding = docs[0]['gte_multi_v768']

embedding[:3]

[-0.039117645, 0.062711135, -0.060027212]

In [20]:
result = impresso.experiments.execute(
    experiment_id="subdoc-embeddings",
    body={
        "solrPayload": {
            "query": "{!knn f=gte_multi_v768 topK=3}" + str(embedding),
            "limit": 3,
            "params": {
                "fq": "type_s:c",   # type_s:c restricts search to chunks! (c=chunk)
                # "fl": "id,score,content_txt_fr,ci_id_s", -- add these later if you want to return only specific fields
                # for now let's return everything
                "hl": False
            }
        }
    }
)

docs = result["solrResponse"]["response"]["docs"]
print(f"✅ Got {len(docs)} Solr document(s)\n")

✅ Got 3 Solr document(s)



In [21]:
for i, d in enumerate(docs):
    print(f"--- Result {i} ---")
    print(d.get("content_txt_fr", "[No text]"))
    print(d.get("content_txt_de", "[No text]"))

--- Result 0 ---
[No text]
gskosten Briefbeföid «.auf die eigentliche Beförderung zwei Drittel aber auf den den Schalterdienst « nd Bestelldienst fallen.Der große Gedanke macht « Schule : er iibec » schritt die Grenzen des Landes ud n versuchte sich auch auf internationalem Boden.Nebst dem Weltpostverein, der uns im Jahre 1875 das internationale Einheitsporto für Briefe brachte, entstanden eine Anzahl Sonderpost der.eine, die Hauptsächlich den Zweck hatten, d « Inlandsporto der Vertragsstaaten auch auf ihren ieg « nseitiaen Auhenveikehr auszudehnen.Selbst uor dem Meere macht « das Einheitsporlo « ich » Halt.Im Jahre 1898 Antrag führte England auf den seines Abgeordneten Henniker-Heaton im Verkehr mit seinen Kolonien das Penny.Porto ein ; Deutschland folgte diesem Beispiele 1N99. Seit 19N8 19l ) 9 verkehrt Großbritannien unö seit auch Deutschland mit den Vereinigten Staaten von Amerika zum Inlandssahe.Begehren Auch in der Schweiz wurden schon laut, öfters die ähnliche Abkommen mit unser

## Entities profiles experiment

In [22]:
result = impresso.experiments.execute(
  experiment_id="entity-profiles",
  body={
    "solrPayload": {
      "query": "wiki_url_s:*Simone*de*Beauvoir*",
      "limit": 1,
      "params": {
        "hl": False
      }
    }
  }
)
entity_doc = result['solrResponse']['response']['docs'][0]

list(entity_doc.keys())

['id',
 'imp_ids_ss',
 'surfaces_ss',
 'ci_ids_ss',
 'mention_keys_ss',
 'ci_lg_s',
 'wiki_masterlabel_s',
 'wiki_url_s',
 'date_of_birth_dt',
 'date_of_death_dt',
 'wkd_occupations_ss',
 'wkd_occupation_qids_ss',
 'wkd_entity_types_ss',
 'wiki_summaries_t',
 'contexts_ss',
 'entity_mixed_emb_v768',
 'entity_encyc_emb_v768',
 'entity_media_emb_v768',
 '_version_',
 '_root_']

In [23]:
entity_doc['wkd_occupations_ss']

['political philosopher',
 'journalist',
 'novelist',
 'autobiographer',
 'essayist',
 'political activist',
 'diarist',
 'women letter writer',
 'philosopher',
 'literary critic',
 'writer',
 'author',
 'feminist',
 'philosophy teacher']

In [24]:
result = impresso.experiments.execute(
  experiment_id="entity-profiles",
  body={
    "solrPayload": {
      "query": "{!knn f=entity_mixed_emb_v768 topK=5}" + str(entity_doc['entity_mixed_emb_v768']),
      "filter": [
          f"-id:{entity_doc['id']}" # exclude target entity itself
      ],
      "limit": 5,
      "params": {
        "hl": False
      }
    }
  }
)
docs = result['solrResponse']['response']['docs']
for doc in docs:
    print(doc['wiki_url_s'])

https://fr.wikipedia.org/wiki/Hélène_de_Beauvoir
https://fr.wikipedia.org/wiki/Jean_Beauvoir
https://fr.wikipedia.org/wiki/Simone_Chalon
https://fr.wikipedia.org/wiki/Sylvia_Earle
https://fr.wikipedia.org/wiki/Gustave_Simon
